<a href="https://colab.research.google.com/github/kadefue/MoEST/blob/main/Code_Only_MoEST_Modeling_for_Secondary_School_Per_Subjects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import pandas as pd

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!ls '/content/drive/MyDrive/GUIDELINES_TSC_JAN2026/Data Set/csvs/'

'COBET 2017-2025.csv'
 Combined_Primary_Electricity_All_G_NG.csv
 Combined_Primary_Electricity_Govt.csv
 Combined_Primary_ICT_All_G_NG.csv
 Combined_Primary_ICT_Govt.csv
 Combined_Secondary_Electricity_All_G_NG.csv
 Combined_Secondary_Electricity_Govt.csv
 Combined_Secondary_ICT_All_G_NG.csv
 Combined_Secondary_ICT_Govt.csv
 Combined_Secondary_Laboratories_All_G_NG.csv
 Combined_Secondary_Laboratories_Govt.csv
'Data-Pre-Primary Enrollment 2016-2025.csv'
'Data-Primary Enrollment 2016-2025.csv'
'Data-Primary repeaters 2017-2025.csv'
'Data-Primary STD VII Leavers 2017-2025.csv'
'Data-Secondary Enrollment 2016-2025.csv'
'Data-Secondary Tables and chairs 2016-2025.csv'
'Data-Teacher Colleges Enrollment 2016-2025.csv'
'Data-Vocational Enrollment 2024-2025.csv'
'Dropout-Primary 2017-2024.csv'
'Dropout-Secondary  2017-2024.csv'
'LGAs Urban and Rural Status.csv'
'PRE-PRIMARY - DISABALITY 2024-2025.csv'
'Pre-primary GER NA NER 2017-2025.csv'
'PRIMARY DESK 2016-2025.csv'
'PRIMARY - DISABALITY 201

In [ ]:
import os

base_directory = '/content/drive/MyDrive/GUIDELINES_TSC_JAN2026/Data Set/csvs/'

# Get the list of all files from the directory
all_files_in_dir = os.listdir(base_directory)

# Filter for CSV files only and clean up potential hidden files/directories
all_files = [f for f in all_files_in_dir if f.endswith('.csv')]

# Define keywords to exclude
exclude_keywords = ['Primary', 'Textbooks', 'Population', 'Teacher', 'COBET', 'Vocational', 'Enrollment']
exclude_keywords = ['Primary', 'Textbooks', 'Population', 'COBET', 'Vocational', 'Enrollment']


# Filter the files
filtered_files = []
for file_name in all_files:
    if not any(keyword.lower() in file_name.lower() for keyword in exclude_keywords):
        filtered_files.append(file_name)

print("Filtered files:")
for f in filtered_files:
    print(f)

Filtered files:
Dropout-Secondary  2017-2024.csv
Data-Secondary Tables and chairs 2016-2025.csv
Secondary-Re_entry.csv
Secondary - DISABALITY 2020-2025.csv
LGAs Urban and Rural Status.csv
Combined_Secondary_Laboratories_Govt.csv
Combined_Secondary_Laboratories_All_G_NG.csv
Combined_Secondary_ICT_All_G_NG.csv
Combined_Secondary_ICT_Govt.csv
Combined_Secondary_Electricity_All_G_NG.csv
Combined_Secondary_Electricity_Govt.csv
Secondary_students_per_subject.csv


In [ ]:
import pandas as pd
loaded_dataframes = {}

for file_name in filtered_files:
    file_path = f"{base_directory}{file_name}"
    df_name = file_name.replace('.csv', '') # Use file name without extension as dictionary key
    try:
        loaded_dataframes[df_name] = pd.read_csv(file_path)
        print(f"Loaded {df_name} successfully.")
    except Exception as e:
        print(f"Error loading {file_name}: {e}")

print("\nDataFrames loaded into dictionary with keys:")
print(loaded_dataframes.keys())

Loaded Dropout-Secondary  2017-2024 successfully.
Loaded Data-Secondary Tables and chairs 2016-2025 successfully.
Loaded Secondary-Re_entry successfully.
Loaded Secondary - DISABALITY 2020-2025 successfully.
Loaded LGAs Urban and Rural Status successfully.
Loaded Combined_Secondary_Laboratories_Govt successfully.
Loaded Combined_Secondary_Laboratories_All_G_NG successfully.
Loaded Combined_Secondary_ICT_All_G_NG successfully.
Loaded Combined_Secondary_ICT_Govt successfully.
Loaded Combined_Secondary_Electricity_All_G_NG successfully.
Loaded Combined_Secondary_Electricity_Govt successfully.
Loaded Secondary_students_per_subject successfully.

DataFrames loaded into dictionary with keys:
dict_keys(['Dropout-Secondary  2017-2024', 'Data-Secondary Tables and chairs 2016-2025', 'Secondary-Re_entry', 'Secondary - DISABALITY 2020-2025', 'LGAs Urban and Rural Status', 'Combined_Secondary_Laboratories_Govt', 'Combined_Secondary_Laboratories_All_G_NG', 'Combined_Secondary_ICT_All_G_NG', 'Combine

In [ ]:
import re

for df_name, df in loaded_dataframes.items():
    print(f"\n--- Cleaning DataFrame: {df_name} ---")

    # 1. Convert 'object' columns to numeric where applicable
    for col in df.select_dtypes(include='object').columns:
        # Check if the column contains commas, which often indicates numerical data stored as string
        if df[col].astype(str).str.contains(',').any() or df[col].astype(str).str.fullmatch(r'\d+\.?\d*').any():
            # Attempt to convert to numeric after removing commas and other non-numeric characters
            # First, clean the string column: remove commas and trim whitespace
            cleaned_col = df[col].astype(str).str.replace(',', '', regex=False).str.strip()

            # Convert to numeric, coercing errors to NaN
            converted_col = pd.to_numeric(cleaned_col, errors='coerce')

            # If a significant portion could be converted, update the column
            # We check if the number of non-null values increased or stayed the same, and if the dtype changed
            if converted_col.notna().sum() > 0 and converted_col.dtype != object:
                df[col] = converted_col
                print(f"  Converted column '{col}' to numeric type.")

    # 2. Drop 'Unnamed: X' columns that are predominantly null
    unnamed_cols = [col for col in df.columns if re.match(r'Unnamed: \d+', str(col))]
    cols_to_drop = []
    for col in unnamed_cols:
        # Calculate null percentage
        null_percentage = df[col].isnull().sum() / len(df) * 100
        # Drop if more than 90% are null
        if null_percentage > 90:
            cols_to_drop.append(col)

    if cols_to_drop:
        df.drop(columns=cols_to_drop, inplace=True)
        print(f"  Dropped columns: {', '.join(cols_to_drop)}")

    print("Updated DataFrame Info:")
    df.info()



--- Cleaning DataFrame: Dropout-Secondary  2017-2024 ---
Updated DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Year          1472 non-null   int64 
 1   REGION        1472 non-null   object
 2   COUNCIL       1472 non-null   object
 3   Death         1472 non-null   int64 
 4   Indiscipline  1472 non-null   int64 
 5   Pregnancy     1472 non-null   int64 
 6   Truancy       1472 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 80.6+ KB

--- Cleaning DataFrame: Data-Secondary Tables and chairs 2016-2025 ---
  Dropped columns: Unnamed: 6, Unnamed: 7
Updated DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1836 entries, 0 to 1835
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   YEAR              1836 non-null   int64 
 

In [ ]:
import numpy as np

# 1. Define a dictionary of expected column names and their variations
expected_cols_info = {
    'Year': ['Year', 'YEAR', 'Academic Year'],
    'Region': ['Region', 'REGION', 'REGON'],
    'Council': ['Council', 'COUNCIL', 'DISTRICT', 'LGA NAME']
}

# Iterate through each DataFrame
for df_name, df in loaded_dataframes.items():
    print(f"\n--- Checking DataFrame: {df_name} ---")
    discrepancies = []

    # Iterate through conceptual columns
    for conceptual_col, possible_names in expected_cols_info.items():
        found_col_name = None
        for name in possible_names:
            if name in df.columns:
                found_col_name = name
                break

        if found_col_name:
            # Check data type
            current_dtype = df[found_col_name].dtype
            null_count = df[found_col_name].isnull().sum()

            # Expected data types
            if conceptual_col == 'Year':
                # Check for numeric type (int, float)
                if not pd.api.types.is_numeric_dtype(current_dtype):
                    discrepancies.append({
                        'column': found_col_name,
                        'issue': 'Inconsistent Data Type',
                        'expected': 'numeric (int/float)',
                        'actual': current_dtype
                    })
            elif conceptual_col in ['Region', 'Council']:
                # Check for object/string type
                if not pd.api.types.is_object_dtype(current_dtype):
                    discrepancies.append({
                        'column': found_col_name,
                        'issue': 'Inconsistent Data Type',
                        'expected': 'object/string',
                        'actual': current_dtype
                    })

            # Check for null values
            if null_count > 0:
                discrepancies.append({
                    'column': found_col_name,
                    'issue': 'Null Values',
                    'count': null_count
                })
        else:
            discrepancies.append({
                'column': conceptual_col,
                'issue': 'Missing Column',
                'possible_names': possible_names
            })

    if discrepancies:
        print("Discrepancies found:")
        for disc in discrepancies:
            print(f"  - {disc}")
    else:
        print("No discrepancies found for core columns.")


--- Checking DataFrame: Dropout-Secondary  2017-2024 ---
No discrepancies found for core columns.

--- Checking DataFrame: Data-Secondary Tables and chairs 2016-2025 ---
No discrepancies found for core columns.

--- Checking DataFrame: Secondary-Re_entry ---
No discrepancies found for core columns.

--- Checking DataFrame: Secondary - DISABALITY 2020-2025 ---
No discrepancies found for core columns.

--- Checking DataFrame: LGAs Urban and Rural Status ---
Discrepancies found:
  - {'column': 'Year', 'issue': 'Missing Column', 'possible_names': ['Year', 'YEAR', 'Academic Year']}

--- Checking DataFrame: Combined_Secondary_Laboratories_Govt ---
No discrepancies found for core columns.

--- Checking DataFrame: Combined_Secondary_Laboratories_All_G_NG ---
No discrepancies found for core columns.

--- Checking DataFrame: Combined_Secondary_ICT_All_G_NG ---
No discrepancies found for core columns.

--- Checking DataFrame: Combined_Secondary_ICT_Govt ---
No discrepancies found for core column

In [ ]:
df_lga_status = loaded_dataframes['LGAs Urban and Rural Status'].copy()

# Drop the 'Remarks' column if it exists
if 'Remarks' in df_lga_status.columns:
    df_lga_status.drop(columns=['Remarks'], inplace=True)

# Rename columns to standard names
df_lga_status.rename(columns={
    'Region': 'Region',
    'Council': 'Council',
    'Classification': 'LGA_Status'
}, inplace=True)

# Convert 'Region' and 'Council' to uppercase for consistent merging
df_lga_status['Region'] = df_lga_status['Region'].str.upper()
df_lga_status['Council'] = df_lga_status['Council'].str.upper()

print("--- df_lga_status after renaming and cleaning ---")
print("First 5 rows:")
print(df_lga_status.head())
print("\nDataFrame Info:")
df_lga_status.info()

--- df_lga_status after renaming and cleaning ---
First 5 rows:
   Region    Council LGA_Status
0  ARUSHA  ARUSHA CC      Urban
1  ARUSHA     ARUSHA      Rural
2  ARUSHA     KARATU      Rural
3  ARUSHA    LONGIDO      Rural
4  ARUSHA       MERU      Rural

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Region      184 non-null    object
 1   Council     184 non-null    object
 2   LGA_Status  184 non-null    object
dtypes: object(3)
memory usage: 4.4+ KB


In [ ]:
import numpy as np

# 1. Define the expected_cols_info dictionary as previously used
expected_cols_info = {
    'Year': ['Year', 'YEAR', 'Academic Year'],
    'Region': ['Region', 'REGION', 'REGON'],
    'Council': ['Council', 'COUNCIL', 'DISTRICT', 'LGA NAME']
}

# Iterate through each DataFrame in the loaded_dataframes dictionary
for df_name, df in loaded_dataframes.items():
    if df_name == 'LGAs Urban and Rural Status':
        continue # Skip the LGA status DataFrame itself

    print(f"\n--- Processing DataFrame: {df_name} ---")

    # a. Identify the actual column names for 'Region' and 'Council'
    actual_region_col = None
    for r_name in expected_cols_info['Region']:
        if r_name in df.columns:
            actual_region_col = r_name
            break

    actual_council_col = None
    for c_name in expected_cols_info['Council']:
        if c_name in df.columns:
            actual_council_col = c_name
            break

    if actual_region_col and actual_council_col:
        # b. Convert 'Region' and 'Council' values to uppercase for consistency with df_lga_status
        df[actual_region_col] = df[actual_region_col].astype(str).str.upper()
        df[actual_council_col] = df[actual_council_col].astype(str).str.upper()

        # c. Perform a left merge operation with df_lga_status
        try:
            merged_df = pd.merge(df, df_lga_status,
                                 left_on=[actual_region_col, actual_council_col],
                                 right_on=['Region', 'Council'],
                                 how='left',
                                 suffixes=('', '_LGA'))

            # Drop the duplicate 'Region_LGA' and 'Council_LGA' columns if they exist
            if 'Region_LGA' in merged_df.columns: merged_df.drop(columns=['Region_LGA'], inplace=True)
            if 'Council_LGA' in merged_df.columns: merged_df.drop(columns=['Council_LGA'], inplace=True)

            # d. Update the DataFrame in the loaded_dataframes dictionary
            loaded_dataframes[df_name] = merged_df
            # e. Print a confirmation message
            print(f"  Merged 'LGA_Status' into {df_name} using '{actual_region_col}' and '{actual_council_col}'.")
            print(f"  First 5 rows of merged {df_name}:\n{loaded_dataframes[df_name].head()}")
        except Exception as e:
            print(f"  Error merging {df_name}: {e}")
    else:
        print(f"  Skipping merge for {df_name}: Missing one or both of 'Region' and 'Council' columns.")

print("\n--- All relevant DataFrames processed for LGA_Status merge. ---")


--- Processing DataFrame: Dropout-Secondary  2017-2024 ---
  Merged 'LGA_Status' into Dropout-Secondary  2017-2024 using 'REGION' and 'COUNCIL'.
  First 5 rows of merged Dropout-Secondary  2017-2024:
   Year  REGION    COUNCIL  Death  Indiscipline  Pregnancy  Truancy  Region  \
0  2017  ARUSHA     ARUSHA      3             1         24      217  ARUSHA   
1  2017  ARUSHA  ARUSHA CC      8            19         56      150  ARUSHA   
2  2017  ARUSHA     KARATU      4             1         16      278  ARUSHA   
3  2017  ARUSHA    LONGIDO      0             3         26       68  ARUSHA   
4  2017  ARUSHA       MERU      4             3         35      216  ARUSHA   

     Council LGA_Status  
0     ARUSHA      Rural  
1  ARUSHA CC      Urban  
2     KARATU      Rural  
3    LONGIDO      Rural  
4       MERU      Rural  

--- Processing DataFrame: Data-Secondary Tables and chairs 2016-2025 ---
  Merged 'LGA_Status' into Data-Secondary Tables and chairs 2016-2025 using 'REGION' and 'COUN

In [ ]:
import numpy as np
import pandas as pd

# 1. Initialize an empty list to store all findings
all_discrepancies = []

# 2. Define a dictionary of expected column names and their variations
expected_cols_info = {
    'Year': ['Year', 'YEAR', 'Academic Year'],
    'Region': ['Region', 'REGION', 'REGON'],
    'Council': ['Council', 'COUNCIL', 'DISTRICT', 'LGA NAME']
}

# 3. Iterate through each DataFrame
for df_name, df in loaded_dataframes.items():
    # 4. Initialize an empty list for discrepancies specific to the current DataFrame
    discrepancies = []

    # 5. Iterate through conceptual columns
    for conceptual_col, possible_names in expected_cols_info.items():
        found_col_name = None
        for name in possible_names:
            if name in df.columns:
                found_col_name = name
                break

        if found_col_name:
            # Get data type and null count
            current_dtype = df[found_col_name].dtype
            null_count = df[found_col_name].isnull().sum()

            # Check data type consistency
            if conceptual_col == 'Year':
                if not pd.api.types.is_numeric_dtype(current_dtype):
                    discrepancies.append({
                        'dataframe': df_name,
                        'column': found_col_name,
                        'issue': 'Inconsistent Data Type',
                        'expected': 'numeric (int/float)',
                        'actual': str(current_dtype)
                    })
            elif conceptual_col in ['Region', 'Council']:
                if not pd.api.types.is_object_dtype(current_dtype):
                    discrepancies.append({
                        'dataframe': df_name,
                        'column': found_col_name,
                        'issue': 'Inconsistent Data Type',
                        'expected': 'object/string',
                        'actual': str(current_dtype)
                    })

            # Check for null values
            if null_count > 0:
                discrepancies.append({
                    'dataframe': df_name,
                    'column': found_col_name,
                    'issue': 'Null Values',
                    'count': int(null_count)
                })
        else:
            # Record missing column discrepancy
            discrepancies.append({
                'dataframe': df_name,
                'column': conceptual_col,
                'issue': 'Missing Column',
                'possible_names': possible_names
            })

    # 6. If discrepancies found for the DataFrame, add them to the all_discrepancies list
    if discrepancies:
        all_discrepancies.extend(discrepancies)

# 7. Print all inconsistencies found
if all_discrepancies:
    print("--- Overall Discrepancies in Core Columns ---")
    for disc in all_discrepancies:
        print(f"  - {disc}")
else:
    print("No discrepancies found for core columns across all DataFrames.")

--- Overall Discrepancies in Core Columns ---
  - {'dataframe': 'Dropout-Secondary  2017-2024', 'column': 'Region', 'issue': 'Null Values', 'count': 56}
  - {'dataframe': 'Dropout-Secondary  2017-2024', 'column': 'Council', 'issue': 'Null Values', 'count': 56}
  - {'dataframe': 'Secondary - DISABALITY 2020-2025', 'column': 'Region', 'issue': 'Null Values', 'count': 4}
  - {'dataframe': 'Secondary - DISABALITY 2020-2025', 'column': 'Council', 'issue': 'Null Values', 'count': 4}
  - {'dataframe': 'LGAs Urban and Rural Status', 'column': 'Year', 'issue': 'Missing Column', 'possible_names': ['Year', 'YEAR', 'Academic Year']}
  - {'dataframe': 'Combined_Secondary_Laboratories_Govt', 'column': 'Region', 'issue': 'Null Values', 'count': 66}
  - {'dataframe': 'Combined_Secondary_Laboratories_Govt', 'column': 'Council', 'issue': 'Null Values', 'count': 66}
  - {'dataframe': 'Combined_Secondary_Laboratories_All_G_NG', 'column': 'Region', 'issue': 'Null Values', 'count': 68}
  - {'dataframe': 'Co

In [ ]:
print("DataFrames before removing 'LGAs Urban and Rural Status':")
print(loaded_dataframes.keys())

# Remove the 'LGAs Urban and Rural Status' DataFrame
if 'LGAs Urban and Rural Status' in loaded_dataframes:
    del loaded_dataframes['LGAs Urban and Rural Status']
    print("\nSuccessfully removed 'LGAs Urban and Rural Status' DataFrame.")
else:
    print("\n'LGAs Urban and Rural Status' DataFrame not found (already removed or never existed).")

print("\nDataFrames after removing 'LGAs Urban and Rural Status':")
print(loaded_dataframes.keys())

DataFrames before removing 'LGAs Urban and Rural Status':
dict_keys(['Dropout-Secondary  2017-2024', 'Data-Secondary Tables and chairs 2016-2025', 'Secondary-Re_entry', 'Secondary - DISABALITY 2020-2025', 'LGAs Urban and Rural Status', 'Combined_Secondary_Laboratories_Govt', 'Combined_Secondary_Laboratories_All_G_NG', 'Combined_Secondary_ICT_All_G_NG', 'Combined_Secondary_ICT_Govt', 'Combined_Secondary_Electricity_All_G_NG', 'Combined_Secondary_Electricity_Govt', 'Secondary_students_per_subject'])

Successfully removed 'LGAs Urban and Rural Status' DataFrame.

DataFrames after removing 'LGAs Urban and Rural Status':
dict_keys(['Dropout-Secondary  2017-2024', 'Data-Secondary Tables and chairs 2016-2025', 'Secondary-Re_entry', 'Secondary - DISABALITY 2020-2025', 'Combined_Secondary_Laboratories_Govt', 'Combined_Secondary_Laboratories_All_G_NG', 'Combined_Secondary_ICT_All_G_NG', 'Combined_Secondary_ICT_Govt', 'Combined_Secondary_Electricity_All_G_NG', 'Combined_Secondary_Electricity_Govt

In [ ]:
print("DataFrames before removal:")
print(loaded_dataframes.keys())

# Remove the 'Pre-primary GER NA NER 2017-2025' DataFrame
if 'Pre-primary GER NA NER 2017-2025' in loaded_dataframes:
    del loaded_dataframes['Pre-primary GER NA NER 2017-2025']
    print("\nSuccessfully removed 'Pre-primary GER NA NER 2017-2025' DataFrame.")
else:
    print("\n'Pre-primary GER NA NER 2017-2025' DataFrame not found (already removed or never existed).")

print("\nDataFrames after removal:")
print(loaded_dataframes.keys())

DataFrames before removal:
dict_keys(['Dropout-Secondary  2017-2024', 'Data-Secondary Tables and chairs 2016-2025', 'Secondary-Re_entry', 'Secondary - DISABALITY 2020-2025', 'Combined_Secondary_Laboratories_Govt', 'Combined_Secondary_Laboratories_All_G_NG', 'Combined_Secondary_ICT_All_G_NG', 'Combined_Secondary_ICT_Govt', 'Combined_Secondary_Electricity_All_G_NG', 'Combined_Secondary_Electricity_Govt', 'Secondary_students_per_subject'])

'Pre-primary GER NA NER 2017-2025' DataFrame not found (already removed or never existed).

DataFrames after removal:
dict_keys(['Dropout-Secondary  2017-2024', 'Data-Secondary Tables and chairs 2016-2025', 'Secondary-Re_entry', 'Secondary - DISABALITY 2020-2025', 'Combined_Secondary_Laboratories_Govt', 'Combined_Secondary_Laboratories_All_G_NG', 'Combined_Secondary_ICT_All_G_NG', 'Combined_Secondary_ICT_Govt', 'Combined_Secondary_Electricity_All_G_NG', 'Combined_Secondary_Electricity_Govt', 'Secondary_students_per_subject'])


In [ ]:
import numpy as np

# 1. Define a dictionary of expected column names and their variations
expected_cols_info = {
    'Region': ['Region', 'REGION', 'REGON'],
    'Council': ['Council', 'COUNCIL', 'DISTRICT', 'LGA NAME']
}

# 2. Iterate through each DataFrame in the loaded_dataframes dictionary
for df_name, df in loaded_dataframes.items():
    # 3. Skip the 'Pre-primary GER NA NER 2017-2025' DataFrame
    if df_name == 'Pre-primary GER NA NER 2017-2025':
        print(f"\n--- Skipping DataFrame: {df_name} (already processed) ---")
        continue

    print(f"\n--- Processing DataFrame: {df_name} ---")

    # 4. Identify the actual column names for 'Region' and 'Council'
    actual_region_col = None
    for r_name in expected_cols_info['Region']:
        if r_name in df.columns:
            actual_region_col = r_name
            break

    actual_council_col = None
    for c_name in expected_cols_info['Council']:
        if c_name in df.columns:
            actual_council_col = c_name
            break

    # 5. Drop rows with null values in the identified 'Region' or 'Council' columns
    if actual_region_col and actual_council_col:
        initial_rows = len(df)
        df.dropna(subset=[actual_region_col, actual_council_col], inplace=True)
        rows_dropped = initial_rows - len(df)

        # Reassign the modified DataFrame back to loaded_dataframes[df_name]
        loaded_dataframes[df_name] = df

        # 6. Print confirmation message
        if rows_dropped > 0:
            print(f"  Dropped {rows_dropped} rows due to null values in '{actual_region_col}' or '{actual_council_col}'.")
        else:
            print(f"  No null values found in '{actual_region_col}' or '{actual_council_col}'. No rows dropped.")

    else:
        print(f"  Skipping row drop for {df_name}: Missing one or both of 'Region' and 'Council' columns.")



--- Processing DataFrame: Dropout-Secondary  2017-2024 ---
  Dropped 56 rows due to null values in 'Region' or 'Council'.

--- Processing DataFrame: Data-Secondary Tables and chairs 2016-2025 ---
  No null values found in 'Region' or 'Council'. No rows dropped.

--- Processing DataFrame: Secondary-Re_entry ---
  No null values found in 'Region' or 'Council'. No rows dropped.

--- Processing DataFrame: Secondary - DISABALITY 2020-2025 ---
  Dropped 4 rows due to null values in 'Region' or 'Council'.

--- Processing DataFrame: Combined_Secondary_Laboratories_Govt ---
  Dropped 66 rows due to null values in 'Region' or 'Council'.

--- Processing DataFrame: Combined_Secondary_Laboratories_All_G_NG ---
  Dropped 68 rows due to null values in 'Region' or 'Council'.

--- Processing DataFrame: Combined_Secondary_ICT_All_G_NG ---
  Dropped 99 rows due to null values in 'Region' or 'Council'.

--- Processing DataFrame: Combined_Secondary_ICT_Govt ---
  Dropped 75 rows due to null values in 'Reg

In [ ]:
import numpy as np
import pandas as pd

# 1. Define column mapping
core_columns_mapping = {
    'Year': ['Year', 'YEAR', 'Academic Year'],
    'Region': ['Region', 'REGION', 'REGON'],
    'Council': ['Council', 'COUNCIL', 'DISTRICT', 'LGA NAME']
}

dataframe_features = {}
summary_data = []

for df_name, df in loaded_dataframes.items():
    current_core_cols = []

    # 2. Identify Core Columns using a concise loop
    for conceptual_col, variations in core_columns_mapping.items():
        # Find the first variation that exists in the dataframe
        match = next((v for v in variations if v in df.columns), None)
        if match:
            current_core_cols.append(match)

    # 3. Identify Numerical Metrics
    # We select numeric types and immediately exclude any that are in current_core_cols
    all_numeric = df.select_dtypes(include=[np.number]).columns.tolist()
    current_numerical_metrics = [col for col in all_numeric if col not in current_core_cols]

    # 4. Store features
    dataframe_features[df_name] = {
        'core_columns': current_core_cols,
        'numerical_metrics': current_numerical_metrics
    }

    # 5. Prep data for the email table
    summary_data.append({
        "DataFrame Name": df_name,
        "Core Columns": ", ".join(current_core_cols),
        "Numerical Metrics": ", ".join(current_numerical_metrics)
    })

# 6. Generate the Table
summary_df = pd.DataFrame(summary_data)
print(summary_df.to_markdown(index=False))


| DataFrame Name                             | Core Columns          | Numerical Metrics                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [ ]:
# @title
!pip install geopy
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time
import os

# Define the filename for cached geodata
GEODATA_FILENAME = '/content/drive/MyDrive/MOEST/tanzania_council_geodata.csv'

# Check if geodata file already exists
if os.path.exists(GEODATA_FILENAME):
    print(f"Loading geodata from existing file: {GEODATA_FILENAME}")
    geo_data_final = pd.read_csv(GEODATA_FILENAME)
    # Recreate geo_data for clustering if needed by the subsequent cell
    geo_data = geo_data_final.copy()
else:
    # 1. CONSOLIDATE UNIQUE LOCATIONS
    # ---------------------------------------------------------

    # We extract just the Region and Council columns from every dataframe
    location_list = []
    # Iterate over the values (DataFrames) of the loaded_dataframes dictionary
    for df_name, df_data in loaded_dataframes.items():
        # Ensure columns exist to avoid errors (handling case sensitivity if needed)
        # Check for both 'Region' and 'Council' in a case-insensitive manner
        # and get the actual column names if they exist
        region_col = next((col for col in df_data.columns if col.lower() in ['region', 'regon']), None)
        council_col = next((col for col in df_data.columns if col.lower() in ['council', 'district', 'lga name']), None)

        if region_col and council_col:
            # Ensure columns are treated as strings before uppercasing
            subset = df_data[[region_col, council_col]].astype(str).drop_duplicates().copy()
            subset.rename(columns={region_col: 'Region', council_col: 'Council'}, inplace=True)
            subset['Region'] = subset['Region'].str.upper()
            subset['Council'] = subset['Council'].str.upper()
            location_list.append(subset)

    # Concatenate all into one big list and drop duplicates
    geo_data = pd.concat(location_list).drop_duplicates().reset_index(drop=True)

    print(f"Total unique Council locations found: {len(geo_data)}")

    # 2. PREPARE SEARCH QUERIES
    # ---------------------------------------------------------
    # Geocoders work best with full context. We combine Council, Region, and Country.
    # We also clean common abbreviations to help the geocoder (Optional but recommended)
    def clean_council_name(name):
        # Example: 'Ilala MC' -> 'Ilala Municipal Council'
        name = str(name).replace(" MC", " Municipal Council")
        name = name.replace(" DC", " District Council")
        name = name.replace(" TC", " Town Council")
        name = name.replace(" CC", " City Council")
        return name

    geo_data['Search_Query'] = (
        geo_data['Council'].apply(clean_council_name) + ", " +
        geo_data['Region'] + ", Tanzania"
    )

    # 3. GEOCODING (FETCH LAT/LON)
    # ---------------------------------------------------------
    # Initialize the geocoder
    # IMPORTANT: Provide a unique user_agent to comply with Nominatim's usage policy
    geolocator = Nominatim(user_agent="tanzania_education_mapping_project_v1")

    # Use RateLimiter to avoid hitting the API too fast (Nominatim requires 1 sec delay)
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5)

    print("Starting geocoding... this may take a few minutes depending on list size.")

    # Create lists to store results
    lats = []
    longs = []

    for query in geo_data['Search_Query']:
        try:
            location = geocode(query)
            if location:
                lats.append(location.latitude)
                longs.append(location.longitude)
                # Optional: Print success to track progress
                # print(f"Found: {query}")
            else:
                lats.append(None)
                longs.append(None)
                print(f"Not Found: {query}")
        except Exception as e:
            print(f"Error fetching {query}: {e}")
            lats.append(None)
            longs.append(None)

    # 4. FINALIZE DATAFRAME
    # ---------------------------------------------------------
    geo_data['Latitude'] = lats
    geo_data['Longitude'] = longs

    # Drop the temporary search query column
    geo_data_final = geo_data[['Region', 'Council', 'Latitude', 'Longitude']]

    # Remove rows where coordinates were not found (optional)
    # geo_data_final = geo_data_final.dropna(subset=['Latitude', 'Longitude'])

    print("Geocoding complete.")
    print(geo_data_final.head())

    # 5. SAVE TO CSV (Highly Recommended)
    # ---------------------------------------------------------
    # Save immediately so you don't have to re-run the geocoding (it is slow)
    geo_data_final.to_csv(GEODATA_FILENAME, index=False)


Loading geodata from existing file: /content/drive/MyDrive/MOEST/tanzania_council_geodata.csv


In [ ]:
import os

base_directory = '/content/drive/MyDrive/GUIDELINES_TSC_JAN2026/Data Set/csvs/'

# Get the list of all files from the directory
all_files_in_dir = os.listdir(base_directory)

# Filter for CSV files only and clean up potential hidden files/directories
all_files = [f for f in all_files_in_dir if f.endswith('.csv')]

# Define keywords to exclude
exclude_keywords = ['Secondary', 'Textbooks', 'Population', 'Teacher', 'COBET', 'Vocational']

# Filter the files
filtered_files = []
for file_name in all_files:
    if not any(keyword.lower() in file_name.lower() for keyword in exclude_keywords):
        filtered_files.append(file_name)

print("Filtered files:")
for f in filtered_files:
    print(f)

Filtered files:
Data-Pre-Primary Enrollment 2016-2025.csv
Data-Primary Enrollment 2016-2025.csv
Dropout-Primary 2017-2024.csv
Pre-primary GER NA NER 2017-2025.csv
Data-Primary repeaters 2017-2025.csv
Data-Primary STD VII Leavers 2017-2025.csv
Primary GIR NA NIR 2017-2025.csv
PRIMARY DESK 2016-2025.csv
Primary-Re_entry.csv
PRE-PRIMARY - DISABALITY 2024-2025.csv
PRIMARY - DISABALITY 2017-2025.csv
PRIMARY Pit Latrine AND CLASSROOMS  Final 2016-2025.csv
LGAs Urban and Rural Status.csv
Combined_Primary_ICT_Govt.csv
Combined_Primary_ICT_All_G_NG.csv
Combined_Primary_Electricity_All_G_NG.csv
Combined_Primary_Electricity_Govt.csv


In [ ]:
from sklearn.cluster import KMeans
import pandas as pd

# Ensure Latitude and Longitude are numeric and handle nulls
geo_data['Latitude'] = pd.to_numeric(geo_data['Latitude'], errors='coerce')
geo_data['Longitude'] = pd.to_numeric(geo_data['Longitude'], errors='coerce')
geo_data.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Select the coordinates for clustering
X = geo_data[['Latitude', 'Longitude']]

# Instantiate KMeans model
n_clusters = 5 # Example: choosing 5 clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10) # n_init for modern scikit-learn versions

# Fit the model and add cluster labels to geo_data
geo_data['Geo_Cluster'] = kmeans.fit_predict(X)

print("--- geo_data after adding Geo_Cluster column ---")
print("First 5 rows with Latitude, Longitude, and Geo_Cluster:")
print(geo_data.head())
print("\nDataFrame Info:")
geo_data.info()

--- geo_data after adding Geo_Cluster column ---
First 5 rows with Latitude, Longitude, and Geo_Cluster:
   Region    Council  Latitude  Longitude  Unnamed: 4  Geo_Cluster
0  ARUSHA  ARUSHA CC -3.370630  36.695839         NaN            3
1  ARUSHA     ARUSHA -3.369683  36.688079         NaN            3
2  ARUSHA     KARATU -3.490197  35.568288         NaN            3
3  ARUSHA    LONGIDO -2.644670  36.424087         NaN            3
4  ARUSHA       MERU -3.325338  36.904361         NaN            3

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Region       187 non-null    object 
 1   Council      187 non-null    object 
 2   Latitude     187 non-null    float64
 3   Longitude    187 non-null    float64
 4   Unnamed: 4   0 non-null      float64
 5   Geo_Cluster  187 non-null    int32  
dtypes: float64(3), int32(1), object

In [ ]:
import numpy as np

# 1. Initialize an expected_cols_info dictionary
expected_cols_info = {
    'Region': ['Region', 'REGION', 'REGON'],
    'Council': ['Council', 'COUNCIL', 'DISTRICT', 'LGA NAME']
}

# Iterate through each DataFrame in the loaded_dataframes dictionary
for df_name, df in loaded_dataframes.items():
    print(f"\n--- Processing DataFrame for Geo_Cluster merge: {df_name} ---")

    # 3. For each DataFrame, identify the actual column names for 'Region' and 'Council'
    actual_region_col = None
    for r_name in expected_cols_info['Region']:
        if r_name in df.columns:
            actual_region_col = r_name
            break

    actual_council_col = None
    for c_name in expected_cols_info['Council']:
        if c_name in df.columns:
            actual_council_col = c_name
            break

    if actual_region_col and actual_council_col:
        # 4. Convert the values in the identified 'Region' and 'Council' columns of the current DataFrame to uppercase
        df[actual_region_col] = df[actual_region_col].astype(str).str.upper()
        df[actual_council_col] = df[actual_council_col].astype(str).str.upper()

        # 5. Perform a left merge operation between the current DataFrame and geo_data.
        # 6. Use the actual 'Region' and 'Council' column names from the current DataFrame and 'Region' and 'Council' from geo_data as the merge keys.
        try:
            merged_df = pd.merge(df, geo_data[['Region', 'Council', 'Geo_Cluster']],
                                 left_on=[actual_region_col, actual_council_col],
                                 right_on=['Region', 'Council'],
                                 how='left',
                                 suffixes=('', '_geo'))

            # 7. After the merge, if any duplicate 'Region' or 'Council' columns are created, drop them
            if 'Region_geo' in merged_df.columns: merged_df.drop(columns=['Region_geo'], inplace=True)
            if 'Council_geo' in merged_df.columns: merged_df.drop(columns=['Council_geo'], inplace=True)

            # 8. Update the current DataFrame in the loaded_dataframes dictionary
            loaded_dataframes[df_name] = merged_df
            # 9. Print a confirmation message
            print(f"  Successfully merged 'Geo_Cluster' into {df_name} using '{actual_region_col}' and '{actual_council_col}'.")
        except Exception as e:
            print(f"  Error merging Geo_Cluster into {df_name}: {e}")
    else:
        print(f"  Skipping Geo_Cluster merge for {df_name}: Missing one or both of 'Region' and 'Council' columns.")

print("\n--- All relevant DataFrames processed for Geo_Cluster merge. ---")


--- Processing DataFrame for Geo_Cluster merge: Dropout-Secondary  2017-2024 ---
  Successfully merged 'Geo_Cluster' into Dropout-Secondary  2017-2024 using 'Region' and 'Council'.

--- Processing DataFrame for Geo_Cluster merge: Data-Secondary Tables and chairs 2016-2025 ---
  Successfully merged 'Geo_Cluster' into Data-Secondary Tables and chairs 2016-2025 using 'Region' and 'Council'.

--- Processing DataFrame for Geo_Cluster merge: Secondary-Re_entry ---
  Successfully merged 'Geo_Cluster' into Secondary-Re_entry using 'Region' and 'Council'.

--- Processing DataFrame for Geo_Cluster merge: Secondary - DISABALITY 2020-2025 ---
  Successfully merged 'Geo_Cluster' into Secondary - DISABALITY 2020-2025 using 'Region' and 'Council'.

--- Processing DataFrame for Geo_Cluster merge: Combined_Secondary_Laboratories_Govt ---
  Successfully merged 'Geo_Cluster' into Combined_Secondary_Laboratories_Govt using 'Region' and 'Council'.

--- Processing DataFrame for Geo_Cluster merge: Combined_

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
# Added the requested Scikit-Learn Ensemble models
from sklearn.ensemble import (
    AdaBoostRegressor,
    HistGradientBoostingRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor
)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import re

# ==========================================
# 1. Data Cleaning & Standardization
# ==========================================
def standard_cols(df):
    """Standardizes column names to Upper Case and strips whitespace."""
    df = df.copy()
    # Convert to string first to handle potential non-string headers
    df.columns = [str(c).strip().upper() for c in df.columns]

    # Standardize common location columns
    rename_map = {
        'YEAR': 'YEAR', 'REGION': 'REGION', 'COUNCIL': 'COUNCIL',
        'LGA_STATUS': 'LGA_STATUS', 'GEO_CLUSTER': 'GEO_CLUSTER'
    }
    df.rename(columns=rename_map, inplace=True)

    # Drop duplicates
    df = df.loc[:, ~df.columns.duplicated()]
    return df

def prepare_secondary_datasets(df_subject_enroll, df_drop, df_table, df_reentry, df_disability, df_ict, df_elec, df_labs):
    """
    Cleans and prepares the Subject-Level enrollment data.
    """
    # Standardize all inputs
    enroll = standard_cols(df_subject_enroll)
    drops = standard_cols(df_drop)
    tables = standard_cols(df_table)
    reentry = standard_cols(df_reentry)
    disability = standard_cols(df_disability)
    ict = standard_cols(df_ict)
    elec = standard_cols(df_elec)
    labs = standard_cols(df_labs)

    # --- 1. Reshape Subject Enrollment (Wide to Long) ---
    # Identify ID columns
    id_vars = ['YEAR', 'REGION', 'COUNCIL']
    available_ids = [c for c in id_vars if c in enroll.columns]

    # Identify Subject Columns (Those starting with 'FORM')
    subject_cols = [c for c in enroll.columns if 'FORM ' in c and ' - ' in c]

    print(f"Detected {len(subject_cols)} subject columns to melt.")

    # Melt to Long Format
    enroll_long = enroll.melt(id_vars=available_ids,
                              value_vars=subject_cols,
                              var_name='RAW_COL',
                              value_name='ENROLLMENT')

    # --- 2. Extract Metadata from Column Headers ---
    # Extract Form Number (e.g., "FORM 1" -> 1)
    enroll_long['FORM_NUM'] = enroll_long['RAW_COL'].str.extract(r'FORM (\d)').astype(int)

    # Extract Subject Name (e.g., "PHYSICS_GENERAL")
    enroll_long['SUBJECT'] = enroll_long['RAW_COL'].str.split(' - ').str[1].str.strip()

    # Drop the raw column to save memory
    enroll_long.drop(columns=['RAW_COL'], inplace=True)

    # --- 3. Aggregations for Feature Sets ---
    keys = ['YEAR', 'REGION', 'COUNCIL']

    # Re-entry
    re_cols = [c for c in reentry.columns if 'RE-ENROLLED' in c]
    if re_cols:
        reentry['TOTAL_REENTRY'] = reentry[re_cols].sum(axis=1)
        reentry_clean = reentry[keys + ['TOTAL_REENTRY']].groupby(keys).sum().reset_index()
    else:
        reentry_clean = pd.DataFrame(columns=keys + ['TOTAL_REENTRY'])

    # Disability
    dis_cols = ['BLIND', 'LOW VISION']
    avail_dis = [c for c in dis_cols if c in disability.columns]
    if avail_dis:
        disability['TOTAL_DISABLED'] = disability[avail_dis].sum(axis=1)
        dis_clean = disability[keys + ['TOTAL_DISABLED']].groupby(keys).sum().reset_index()
    else:
        dis_clean = pd.DataFrame(columns=keys + ['TOTAL_DISABLED'])

    # Tables
    if 'AVAILABLE_TABLES' in tables.columns:
        tables_clean = tables[keys + ['AVAILABLE_TABLES']].copy()
    else:
        tables_clean = pd.DataFrame(columns=keys + ['AVAILABLE_TABLES'])

    # ICT
    if 'DESKTOP COMPUTERS' in ict.columns and 'LAPTOP COMPUTERS' in ict.columns:
        ict['TOTAL_COMPUTERS'] = ict['DESKTOP COMPUTERS'].fillna(0) + ict['LAPTOP COMPUTERS'].fillna(0)
        ict_clean = ict[keys + ['TOTAL_COMPUTERS']].copy()
    else:
        ict_clean = pd.DataFrame(columns=keys + ['TOTAL_COMPUTERS'])

    # Electricity
    elec_col = 'NATIONAL GRID (TANESCO) %'
    if elec_col in elec.columns:
        elec_clean = elec[keys + [elec_col]].copy()
        elec_clean.rename(columns={elec_col: 'ELEC_GRID_PCT'}, inplace=True)
    else:
        elec_clean = pd.DataFrame(columns=keys + ['ELEC_GRID_PCT'])

    # Labs
    lab_types = ['BIOLOGY', 'CHEMISTRY', 'PHYSICS']
    lab_cols = [f'{l} LABORATORY AVAILABLE' for l in lab_types]
    valid_labs = [c for c in lab_cols if c in labs.columns]
    if valid_labs:
        labs['TOTAL_LABS'] = labs[valid_labs].sum(axis=1)
        labs_clean = labs[keys + ['TOTAL_LABS']].copy()
    else:
        labs_clean = pd.DataFrame(columns=keys + ['TOTAL_LABS'])

    return enroll_long, drops, tables_clean, reentry_clean, dis_clean, ict_clean, elec_clean, labs_clean

# ==========================================
# 2. Feature Engineering (Subject-Aware)
# ==========================================
def engineer_secondary_features(df, drops, tables, reentry, disability, ict, elec, labs):
    keys = ['YEAR', 'REGION', 'COUNCIL']

    # --- Merge Infrastructure (Council Level) ---
    df = df.merge(tables, on=keys, how='left').fillna({'AVAILABLE_TABLES': 0})
    df = df.merge(reentry, on=keys, how='left').fillna({'TOTAL_REENTRY': 0})
    df = df.merge(disability, on=keys, how='left').fillna({'TOTAL_DISABLED': 0})
    df = df.merge(ict, on=keys, how='left').fillna({'TOTAL_COMPUTERS': 0})
    df = df.merge(elec, on=keys, how='left').fillna({'ELEC_GRID_PCT': 0})
    df = df.merge(labs, on=keys, how='left').fillna({'TOTAL_LABS': 0})

    # Dropouts
    drop_cols = ['TRUANCY', 'PREGNANCY', 'INDISCIPLINE']
    valid_drops = [c for c in drop_cols if c in drops.columns]
    if valid_drops:
        d_grp = drops.groupby(keys)[valid_drops].sum().reset_index()
        df = df.merge(d_grp, on=keys, how='left').fillna(0)

    # Sort for Time Series Ops
    df = df.sort_values(['REGION', 'COUNCIL', 'SUBJECT', 'FORM_NUM', 'YEAR'])

    # --- Temporal Features ---
    df['IS_ELECTION_YEAR'] = df['YEAR'].apply(lambda x: 1 if x in [2015, 2020, 2025] else 0)

    # --- Subject-Specific Lags & Trends ---
    g = df.groupby(['REGION', 'COUNCIL', 'SUBJECT', 'FORM_NUM'])
    df['LAG_1'] = g['ENROLLMENT'].shift(1)
    df['LAG_2'] = g['ENROLLMENT'].shift(2)
    df['YOY_GROWTH'] = (df['ENROLLMENT'] - df['LAG_1']) / (df['LAG_1'] + 1e-5)

    # --- Subject-Specific Cohort Logic ---
    df['PREV_YEAR'] = df['YEAR'] - 1
    df['PREV_FORM_NUM'] = df['FORM_NUM'] - 1

    df['LOOKUP_KEY'] = (
        df['YEAR'].astype(str) + '_' +
        df['REGION'] + '_' +
        df['COUNCIL'] + '_' +
        df['SUBJECT'] + '_' +
        df['FORM_NUM'].astype(str)
    )

    lookup_df = df.groupby('LOOKUP_KEY')['ENROLLMENT'].sum()
    enrollment_lookup = lookup_df.to_dict()

    df['COHORT_LOOKUP_KEY'] = (
        df['PREV_YEAR'].astype(str) + '_' +
        df['REGION'] + '_' +
        df['COUNCIL'] + '_' +
        df['SUBJECT'] + '_' +
        df['PREV_FORM_NUM'].astype(str)
    )

    df['COHORT_LAG'] = df['COHORT_LOOKUP_KEY'].map(enrollment_lookup)
    df['COHORT_LAG'] = df['COHORT_LAG'].fillna(-1)

    df.drop(columns=['PREV_YEAR', 'PREV_FORM_NUM', 'LOOKUP_KEY', 'COHORT_LOOKUP_KEY'], inplace=True)
    df = df.fillna(-1)

    return df

# ==========================================
# 3. Model Training (Updated for Subject)
# ==========================================
def calculate_metrics(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    # Handle division by zero for MAPE
    mask = y_true != 0
    if mask.sum() > 0:
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        mape = np.nan

    print(f"--- {model_name} Results ---")
    print(f"MAE  : {mae:,.2f}")
    print(f"RMSE : {rmse:,.2f}")
    print(f"R^2  : {r2:.4f}")
    print(f"MAPE : {mape:.2f}% Accuracy :{(100-mape) if not np.isnan(mape) else 0:.2f}%")
    print("-" * 30)
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'MAPE': mape}

def train_secondary_models(df):
    le = LabelEncoder()
    # Encode Categoricals
    df['REGION_ENC'] = le.fit_transform(df['REGION'].astype(str))
    df['COUNCIL_ENC'] = le.fit_transform(df['COUNCIL'].astype(str))
    df['SUBJECT_ENC'] = le.fit_transform(df['SUBJECT'].astype(str))

    features = [
        'YEAR', 'REGION_ENC', 'COUNCIL_ENC', 'SUBJECT_ENC', 'FORM_NUM',
        'AVAILABLE_TABLES', 'TOTAL_REENTRY', 'TOTAL_DISABLED',
        'TOTAL_COMPUTERS', 'ELEC_GRID_PCT', 'TOTAL_LABS',
        'TRUANCY', 'PREGNANCY', 'INDISCIPLINE',
        'LAG_1', 'LAG_2', 'YOY_GROWTH', 'COHORT_LAG', 'IS_ELECTION_YEAR'
    ]

    features = [f for f in features if f in df.columns]

    # Split Train/Test
    train = df[df['YEAR'] < 2024]
    test = df[df['YEAR'] >= 2024]

    X_train, y_train = train[features], train['ENROLLMENT']
    X_test, y_test = test[features], test['ENROLLMENT']

    print(f"Training on {X_train.shape[0]} rows (Subjects x Locations), Testing on {X_test.shape[0]} rows.")

    results = {}

    # 1. XGBoost
    xgb_model = xgb.XGBRegressor(n_estimators=300, max_depth=9, learning_rate=0.05, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    results['XGB'] = calculate_metrics(y_test, xgb_model.predict(X_test), "XGBoost")

    # 2. LightGBM
    lgb_model = lgb.LGBMRegressor(n_estimators=500, num_leaves=50, min_child_samples=10, learning_rate=0.1, verbose=-1)
    lgb_model.fit(X_train, y_train)
    results['LGBM'] = calculate_metrics(y_test, lgb_model.predict(X_test), "LightGBM")

    # 3. HistGradientBoosting (Scikit-Learn implementation of LightGBM)
    # Excellent for large datasets, handles NaNs natively
    hgb_model = HistGradientBoostingRegressor(max_iter=500, learning_rate=0.1, max_depth=10)
    hgb_model.fit(X_train, y_train)
    results['HistGradBoost'] = calculate_metrics(y_test, hgb_model.predict(X_test), "HistGradientBoosting")

    # 4. GradientBoosting (Standard)
    # Slower, but often very accurate. Reduced estimators to keep training time reasonable.
    gb_model = GradientBoostingRegressor(n_estimators=300, max_depth=9, learning_rate=0.05)
    gb_model.fit(X_train, y_train)
    results['GradientBoost'] = calculate_metrics(y_test, gb_model.predict(X_test), "GradientBoosting")

    # 5. Random Forest
    # Robust baseline. Uses n_jobs=-1 to parallelize.
    rf_model = RandomForestRegressor(n_estimators=300, max_depth=12, n_jobs=-1, random_state=42)
    rf_model.fit(X_train, y_train)
    results['RandomForest'] = calculate_metrics(y_test, rf_model.predict(X_test), "RandomForest")

    return results, xgb_model

# ==========================================
# 4. Pipeline Execution
# ==========================================
def run_secondary_pipeline(df_subject, drop_df, table_df, reentry_df, dis_df, ict_df, elec_df, labs_df):
    print("1. Preparing Subject Datasets...")
    enroll, drops, tables, reentry, dis, ict, elec, labs = prepare_secondary_datasets(
        df_subject, drop_df, table_df, reentry_df, dis_df, ict_df, elec_df, labs_df
    )

    print("2. Engineering Features (Subject Cohorts)...")
    final_df = engineer_secondary_features(enroll, drops, tables, reentry, dis, ict, elec, labs)

    print("\n==========================================")
    print(" SUBJECT-LEVEL ENROLLMENT FORECASTING ")
    print("==========================================")
    metrics, model = train_secondary_models(final_df)

    return final_df, metrics, model

# EXECUTION
# Ensure your dataframe names match exactly what is loaded in your environment
secondary_final_df, sec_metrics, sec_model = run_secondary_pipeline(
    loaded_dataframes["Secondary_students_per_subject"],
    loaded_dataframes["Dropout-Secondary  2017-2024"],
    loaded_dataframes["Data-Secondary Tables and chairs 2016-2025"],
    loaded_dataframes["Secondary-Re_entry"],
    loaded_dataframes["Secondary - DISABALITY 2020-2025"],
    loaded_dataframes["Combined_Secondary_ICT_All_G_NG"],
    loaded_dataframes["Combined_Secondary_Electricity_All_G_NG"],
    loaded_dataframes["Combined_Secondary_Laboratories_All_G_NG"]
)

1. Preparing Subject Datasets...
Detected 283 subject columns to melt.
2. Engineering Features (Subject Cohorts)...

 SUBJECT-LEVEL ENROLLMENT FORECASTING 
Training on 415444 rows (Subjects x Locations), Testing on 104144 rows.
--- XGBoost Results ---
MAE  : 29.79
RMSE : 331.10
R^2  : 0.9640
MAPE : 5.80% Accuracy :94.20%
------------------------------
--- LightGBM Results ---
MAE  : 23.80
RMSE : 295.75
R^2  : 0.9713
MAPE : 6.12% Accuracy :93.88%
------------------------------
--- HistGradientBoosting Results ---
MAE  : 60.66
RMSE : 402.13
R^2  : 0.9469
MAPE : 31.97% Accuracy :68.03%
------------------------------
--- GradientBoosting Results ---
MAE  : 27.23
RMSE : 334.44
R^2  : 0.9632
MAPE : 3.93% Accuracy :96.07%
------------------------------
--- RandomForest Results ---
MAE  : 29.61
RMSE : 342.88
R^2  : 0.9614
MAPE : 7.92% Accuracy :92.08%
------------------------------


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ==========================================
# 1. Data Prep & Feature Engineering (Subject-Aware)
# ==========================================
def prepare_subject_data(df_subject, df_drop, df_table, df_reentry, df_disability, df_ict, df_elec, df_labs):
    # Standardize Column Names
    def clean_cols(d):
        d = d.copy()
        d.columns = [str(c).strip().upper() for c in d.columns]
        rename_map = {'YEAR': 'YEAR', 'REGION': 'REGION', 'COUNCIL': 'COUNCIL'}
        d.rename(columns=rename_map, inplace=True)
        return d.loc[:, ~d.columns.duplicated()]

    enroll = clean_cols(df_subject)
    tables = clean_cols(df_table)
    reentry = clean_cols(df_reentry)
    disability = clean_cols(df_disability)
    ict = clean_cols(df_ict)
    elec = clean_cols(df_elec)
    labs = clean_cols(df_labs)

    # --- Melt to Long Format (Subject Level) ---
    id_vars = ['YEAR', 'REGION', 'COUNCIL']
    # Filter only available ID vars
    id_vars = [c for c in id_vars if c in enroll.columns]

    subject_cols = [c for c in enroll.columns if 'FORM ' in c and ' - ' in c]

    print(f"Melting {len(subject_cols)} subject columns...")
    long_df = enroll.melt(id_vars=id_vars, value_vars=subject_cols, var_name='RAW', value_name='ENROLLMENT')

    # Extract Metadata
    long_df['FORM_NUM'] = long_df['RAW'].str.extract(r'FORM (\d)').astype(int)
    long_df['SUBJECT'] = long_df['RAW'].str.split(' - ').str[1].str.strip()
    long_df.drop(columns=['RAW'], inplace=True)

    # --- Merge Council Features ---
    # (Broadcasting council stats to all subjects in that council)
    keys = ['YEAR', 'REGION', 'COUNCIL']

    # Simple aggregates for merging
    if 'AVAILABLE_TABLES' in tables.columns:
        long_df = long_df.merge(tables[keys + ['AVAILABLE_TABLES']], on=keys, how='left').fillna(0)
    else:
        long_df['AVAILABLE_TABLES'] = 0

    # Labs (Total)
    lab_cols = [c for c in labs.columns if 'LABORATORY' in c]
    if lab_cols:
        labs['TOTAL_LABS'] = labs[lab_cols].sum(axis=1)
        long_df = long_df.merge(labs[keys + ['TOTAL_LABS']], on=keys, how='left').fillna(0)
    else:
        long_df['TOTAL_LABS'] = 0

    # Sort for calculations
    long_df = long_df.sort_values(['REGION', 'COUNCIL', 'SUBJECT', 'FORM_NUM', 'YEAR'])

    # --- Generate History (Lags) ---
    g = long_df.groupby(['REGION', 'COUNCIL', 'SUBJECT', 'FORM_NUM'])
    long_df['LAG_1'] = g['ENROLLMENT'].shift(1)
    long_df['LAG_2'] = g['ENROLLMENT'].shift(2)
    long_df['YOY_GROWTH'] = (long_df['ENROLLMENT'] - long_df['LAG_1']) / (long_df['LAG_1'] + 1e-5)

    # --- Generate Cohort Flow (Subject Specific) ---
    # Form 3 Physics (Year T) needs Form 2 Physics (Year T-1)
    long_df['PREV_YEAR'] = long_df['YEAR'] - 1
    long_df['PREV_FORM'] = long_df['FORM_NUM'] - 1

    # Create Lookup Key
    long_df['LOOKUP_KEY'] = (long_df['YEAR'].astype(str) + '_' + long_df['REGION'] + '_' +
                             long_df['COUNCIL'] + '_' + long_df['SUBJECT'] + '_' + long_df['FORM_NUM'].astype(str))

    lookup = long_df.groupby('LOOKUP_KEY')['ENROLLMENT'].sum().to_dict()

    # Generate Search Key
    long_df['SEARCH_KEY'] = (long_df['PREV_YEAR'].astype(str) + '_' + long_df['REGION'] + '_' +
                             long_df['COUNCIL'] + '_' + long_df['SUBJECT'] + '_' + long_df['PREV_FORM'].astype(str))

    long_df['COHORT_LAG'] = long_df['SEARCH_KEY'].map(lookup).fillna(-1)

    # Clean up
    long_df.drop(columns=['PREV_YEAR', 'PREV_FORM', 'LOOKUP_KEY', 'SEARCH_KEY'], inplace=True)
    long_df['IS_ELECTION_YEAR'] = long_df['YEAR'].isin([2020, 2025, 2030]).astype(int)

    return long_df.fillna(-1)

# ==========================================
# 2. Recursive Forecast Helper (Subject Specific)
# ==========================================
def create_next_year_subject_features(last_year_df, current_year):
    """
    Generates features for Year T based on Year T-1 data for SUBJECTS.
    """
    next_df = last_year_df.copy()
    next_df['YEAR'] = current_year

    # 1. Shift Lags
    next_df['LAG_2'] = next_df['LAG_1']
    next_df['LAG_1'] = next_df['ENROLLMENT'] # The prediction from previous loop becomes history

    # 2. Update Growth (Capped)
    next_df['YOY_GROWTH'] = (next_df['LAG_1'] - next_df['LAG_2']) / (next_df['LAG_2'] + 1e-5)

    # 3. Update Cohort Lag (Subject Specific)
    # We need a dictionary of the T-1 data we just finished processing
    # Key: (Region, Council, Subject, Form) -> Value: Enrollment
    cohort_lookup = last_year_df.set_index(['REGION', 'COUNCIL', 'SUBJECT', 'FORM_NUM'])['ENROLLMENT'].to_dict()

    def get_cohort(row):
        target_form = row['FORM_NUM'] - 1
        if target_form < 1: return -1 # Form 1 has no previous form

        # Look for the SAME Subject in the PREVIOUS Form
        key = (row['REGION'], row['COUNCIL'], row['SUBJECT'], target_form)
        return cohort_lookup.get(key, -1)

    next_df['COHORT_LAG'] = next_df.apply(get_cohort, axis=1)

    # 4. Static/Time Features
    next_df['IS_ELECTION_YEAR'] = 1 if current_year in [2025, 2030] else 0
    next_df['ENROLLMENT'] = np.nan # Reset target

    return next_df

# ==========================================
# 3. Main Training & Forecasting Pipeline
# ==========================================
def run_subject_forecast_system(df_subject, df_drop, df_table, df_reentry, df_disability, df_ict, df_elec, df_labs):

    print("STEP 1: Preparing Subject Data...")
    df = prepare_subject_data(df_subject, df_drop, df_table, df_reentry, df_disability, df_ict, df_elec, df_labs)

    # Encoders
    le_reg = LabelEncoder()
    le_cou = LabelEncoder()
    le_sub = LabelEncoder() # <--- New Subject Encoder

    df['REGION_ENC'] = le_reg.fit_transform(df['REGION'].astype(str))
    df['COUNCIL_ENC'] = le_cou.fit_transform(df['COUNCIL'].astype(str))
    df['SUBJECT_ENC'] = le_sub.fit_transform(df['SUBJECT'].astype(str))

    # Features
    feats = [
        'YEAR', 'REGION_ENC', 'COUNCIL_ENC', 'SUBJECT_ENC', 'FORM_NUM',
        'AVAILABLE_TABLES', 'TOTAL_LABS', 'LAG_1', 'LAG_2',
        'YOY_GROWTH', 'COHORT_LAG', 'IS_ELECTION_YEAR'
    ]

    # Train on History (Up to 2025)
    train_df = df[df['YEAR'] <= 2025].copy()
    X = train_df[feats]
    y = train_df['ENROLLMENT']

    print(f"STEP 2: Training Models on {len(train_df)} rows...")

    # XGBoost
    model_xgb = xgb.XGBRegressor(n_estimators=300, max_depth=9, learning_rate=0.05, n_jobs=-1)
    model_xgb.fit(X, y)

    # LightGBM
    model_lgb = lgb.LGBMRegressor(n_estimators=200, num_leaves=60, verbose=-1)
    model_lgb.fit(X, y)

    print("STEP 3: Recursive Forecasting (2026-2030)...")
    future_preds = []

    # Start with 2025 data as the "Previous Year"
    current_sim = df[df['YEAR'] == 2025].copy()

    for year in range(2026, 2031):
        # Generate Features for Next Year
        next_step = create_next_year_subject_features(current_sim, year)

        # Predict
        X_future = next_step[feats]
        pred_xgb = model_xgb.predict(X_future)
        pred_lgb = model_lgb.predict(X_future)

        # Average Ensemble & Safety Clip
        next_step['ENROLLMENT'] = (pred_xgb + pred_lgb) / 2
        next_step['ENROLLMENT'] = next_step['ENROLLMENT'].apply(lambda x: max(0, x))

        future_preds.append(next_step)

        # Update simulation for next loop
        current_sim = next_step.copy()
        print(f" > {year} Forecast Complete.")

    # Compile Results
    forecast_df = pd.concat(future_preds, ignore_index=True)

    # Select clean output columns
    output_cols = ['YEAR', 'REGION', 'COUNCIL', 'SUBJECT', 'FORM_NUM', 'ENROLLMENT']
    final_output = forecast_df[output_cols].copy()
    final_output['ENROLLMENT'] = final_output['ENROLLMENT'].round(0).astype(int)

    return final_output

# ==========================================
# EXECUTION
# ==========================================
subject_forecast_results = run_subject_forecast_system(
    loaded_dataframes["Secondary_students_per_subject"], # <--- Your Subject DataFrame
    loaded_dataframes["Dropout-Secondary  2017-2024"],
    loaded_dataframes["Data-Secondary Tables and chairs 2016-2025"],
    loaded_dataframes["Secondary-Re_entry"],
    loaded_dataframes["Secondary - DISABALITY 2020-2025"],
    loaded_dataframes["Combined_Secondary_ICT_All_G_NG"],
    loaded_dataframes["Combined_Secondary_Electricity_All_G_NG"],
    loaded_dataframes["Combined_Secondary_Laboratories_All_G_NG"]
)

print("\n--- Sample Subject Forecast (2026-2030) ---")
print(subject_forecast_results.head())

STEP 1: Preparing Subject Data...
Melting 283 subject columns...
STEP 2: Training Models on 519588 rows...
STEP 3: Recursive Forecasting (2026-2030)...
 > 2026 Forecast Complete.
 > 2027 Forecast Complete.
 > 2028 Forecast Complete.
 > 2029 Forecast Complete.
 > 2030 Forecast Complete.

--- Sample Subject Forecast (2026-2030) ---
   YEAR  REGION COUNCIL                         SUBJECT  FORM_NUM  ENROLLMENT
0  2026  ARUSHA  ARUSHA             ACCOUNTANCY_GENERAL         1           0
1  2026  ARUSHA  ARUSHA             ACCOUNTANCY_GENERAL         2           0
2  2026  ARUSHA  ARUSHA             ACCOUNTANCY_GENERAL         3           0
3  2026  ARUSHA  ARUSHA             ACCOUNTANCY_GENERAL         4           0
4  2026  ARUSHA  ARUSHA  ADDITIONAL MATHEMATICS_GENERAL         1           0
